## Import pyspark

In [ ]:
#!/usr/bin/env python
# coding: utf-8
from pyspark import SparkConf, SparkContext
import pyspark

## Mapper

* mapper會一行一行讀取input，傳入function裡的值是line
* input的格式是(**M, i, j, value**)和(**N, j, k, value**)
* MN相乘時，我們會將j相同的值相乘(M : i*j, N : j*k)
* 在第一個mapper當中，我們把key設為j，value設為(**M, i, value**)和(**N, k, value**)
* 放在同一個list裡面後，回傳處理完成的資料格式

In [ ]:
def Mapper(line):
    M = []
    dataValue = line.strip('\n').split(',')
    if(dataValue[0]=='M'):          
        M.append((int(dataValue[2]), ('M', int(dataValue[1]), int(dataValue[3]))))
    elif(dataValue[0]=='N'):
        M.append((int(dataValue[1]), ('N', int(dataValue[2]), int(dataValue[3]))))

    return M

* 第二個mapper則是用來相乘的
* P為MN相乘後的矩陣(P = MN)，所以P的大小為i*k的矩陣
* 我們將(**M, i, value1**)和(**N, k, value2**)處理成(**(i, k), value1*value2**)後回傳

In [ ]:
def Mapper1(line):
    P = []
    
    for i in range(500):
        for k in range(500):
            P.append(((i, k), (line[1][i][2]*line[1][k+500][2])))
            
    return P

## Reducer

* 在此次作業，我們只需將key相同的資料加起來，所以只需要將value相加的reducer即可

In [ ]:
def Reducer(x, y):
    return x+y

# Report

* 讀進input(500input.txt)
* 用第一個mapper將資料轉成(M, i, value)和(N, k, value)
* 用reducer把key一樣的value append到後面
* 因為要用append所以中間先將value轉為list，轉完後用reducer操作list相加即為append

In [ ]:
conf = SparkConf().setAppName("matrix")
sc = SparkContext(conf=conf)
assert sc.version >= '1.5.1'
lines = sc.textFile("500input.txt").flatMap(Mapper).map(lambda nameTuple: (nameTuple[0], [ nameTuple[1] ])).reduceByKey(Reducer)
print(lines.collect())
# print(lines.collect())
# value = lines.combineByKey(to_list, append, extend)
# def to_list(a):
#     return [a]

* 用第二個mapper相乘之後，資料會變成(**(i, k), value**)
* 用reducer把同一個位置(key)的值相加後，即可得到相乘後的矩陣

In [ ]:
result = lines.flatMap(Mapper1).reduceByKey(Reducer)
sort = sorted(result.collect())
print(sort)

* 最後將結果寫上Outputfile.txt就完成了

In [ ]:
f = open('Outputfile.txt', 'w')

for i in range(500*500):
    f.write(str(int(i/500))+','+str(i%500)+','+str(sort[i][1])+'\n')
        
f.close()
sc.stop()